In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data_yahoo/train.csv')

In [6]:
df_test = pd.read_csv('data_yahoo/test.csv',header=None,)

In [7]:
df_test

,0,1,2,3
0,9,What makes friendship click?,How does the spark keep going?,good communication is what does it. Can you m...
1,2,Why does Zebras have stripes?,What is the purpose or those stripes? Who do t...,this provides camouflage - predator vision is ...
2,4,What did the itsy bitsy sipder climb up?,NaN,waterspout
3,4,What is the difference between a Bachelors and...,NaN,One difference between a Bachelors and a Maste...
4,3,Why do women get PMS?,NaN,Premenstrual syndrome (PMS) is a group of symp...
...,...,...,...,...
59995,9,"if you could be any fantasy figure, who would ...",NaN,"The invisible man, I'd be straight into the gi..."
59996,8,Tell me something about life most people don't...,"Do you know anything about life, or words of w...",That there is a hell and everyone thinks their...
59997,3,Why are men always thinking of sex?,NaN,It's wired in our brain
59998,6,est ce que DOMENECH est un entraineur: 1: de f...,NaN,de foot mais pas pour être sélectionneur d'une...


In [9]:
df.shape

(1399999, 4)

In [12]:
df_test['0']

KeyError: '0'